# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch15/ch15_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 패키지 버전 체크

check_packages.py 스크립트에서 로드하기 위해 폴더를 추가합니다:

In [1]:
import sys

# 코랩의 경우 깃허브 저장소로부터 python_environment_check.py를 다운로드 합니다.
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
else:
    sys.path.insert(0, '..')

--2023-09-05 05:37:19--  https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1629 (1.6K) [text/plain]
Saving to: ‘python_environment_check.py’

python_environment_ 100%[===================>]   1.59K  --.-KB/s    in 0s      

2023-09-05 05:37:20 (34.0 MB/s) - ‘python_environment_check.py’ saved [1629/1629]



권장 패키지 버전을 확인하세요:

In [2]:
from python_environment_check import check_packages


d = {
    'torch': '1.8.0',
}
check_packages(d)

[OK] Your Python version is 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
[OK] torch 2.0.1+cu118


# 15장 - 순환 신경망으로 순차 데이터 모델링 (파트 1/3)

**목차**

- 순차 데이터 소개
  - 순차 데이터 모델링: 순서를 고려한다
  - 순차 데이터 vs 시계열 데이터
  - 시퀀스 표현
  - 시퀀스 모델링의 종류
- 시퀀스 모델링을 위한 RNN
  - RNN의 반복 구조 이해
  - RNN의 활성화 출력 계산
  - 은닉 순환과 출력 순환
  - 긴 시퀀스 학습의 어려움
  - LSTM 셀

In [3]:
from IPython.display import Image
%matplotlib inline

# 순차 데이터 소개

## 순차 데이터 모델링: 순서를 고려한다

## 순차 데이터 vs 시계열 데이터

## 시퀀스 표현

In [4]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_01.png', width=500)

## 시퀀스 모델링의 종류

In [5]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_02.png', width=500)

# 시퀀스 모델링을 위한 RNN

## RNN의 반복 구조 이해

In [6]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_03.png', width=500)

In [7]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_04.png', width=500)

## RNN의 활성화 출력 계산

In [8]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_05.png', width=500)

In [9]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_06.png', width=700)

## 은닉 순환과 출력 순환

In [10]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_07.png', width=500)

In [11]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh 크기:', w_xh.shape)
print('W_hh 크기:', w_hh.shape)
print('b_xh 크기:', b_xh.shape)
print('b_hh 크기:', b_hh.shape)

W_xh 크기: torch.Size([2, 5])
W_hh 크기: torch.Size([2, 2])
b_xh 크기: torch.Size([2])
b_hh 크기: torch.Size([2])


In [16]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## 간단한 RNN의 출력:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## 수동으로 출력 계산
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'타임 스텝 {t} =>')
    print('   입력           :', xt.numpy())

    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print('   은닉           :', ht.detach().numpy())

    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   출력 (수동)     :', ot.detach().numpy())
    print('   RNN 출력       :', output[:, t].detach().numpy())
    print()

타임 스텝 0 =>
   입력           : [[1. 1. 1. 1. 1.]]
   은닉           : [[-0.4701929   0.58639044]]
   출력 (수동)     : [[-0.3519801   0.52525216]]
   RNN 출력       : [[-0.35198015  0.52525216]]

타임 스텝 1 =>
   입력           : [[2. 2. 2. 2. 2.]]
   은닉           : [[-0.88883156  1.2364398 ]]
   출력 (수동)     : [[-0.68424344  0.76074266]]
   RNN 출력       : [[-0.68424344  0.76074266]]

타임 스텝 2 =>
   입력           : [[3. 3. 3. 3. 3.]]
   은닉           : [[-1.3074702  1.8864892]]
   출력 (수동)     : [[-0.8649416  0.9046636]]
   RNN 출력       : [[-0.8649416   0.90466356]]



## 긴 시퀀스 학습의 어려움

In [13]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_08.png', width=500)

## LSTM 셀

In [14]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_09.png', width=500)